In [2]:
import numpy as np
    
def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

def rank_basescore(base_score, draw_ratio):
    # 计算最大值和最小值的差值
    score_range = max(base_score) - min(base_score)
    # 计算平序的阈值
    draw_gap = draw_ratio * score_range

    # 对列表进行排序并保留原始索引
    indexed_scores = list(enumerate(base_score))
    indexed_scores.sort(key=lambda x: x[1])

    # 处理平序
    ranks = [0] * len(base_score)
    current_rank = 1
    for i in range(len(indexed_scores)):
        if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:
            ranks[indexed_scores[i][0]] = current_rank
        else:
            current_rank = i + 1
            ranks[indexed_scores[i][0]] = current_rank

    return ranks

In [3]:
dimension = 'color'

In [4]:
import json
jsonpath = '../Human_anno/{}.json'.format(dimension)
with open(jsonpath,'r') as f:
    oc = json.load(f)

# history ="../GPT4o_eval_results/{}_gpt4eval_results.json".format(dimension)
# with open(history,'r') as f:
#     gpt4o_eval_history = json.load(f)

In [5]:
models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']
# models = ['videocrafter2', 'pika', 'show1', 'lavie']
# models = ['cogvideox5b','gen3', 'kling']
idexls = []
for i in range(0,len(oc)):
    idexls.append(i)
# for i in range(1,len(oc),3):
#     idexls.append(i)
length = len(idexls)

In [6]:
flag = 0
iternum = 0
randrtio = 0.1
while(flag == 0):
    anno1s = []
    anno2s = []
    anno3s = []
    anno4s = []
    iternum += 1
    gptvsanno1_spearman = np.zeros(length)
    gptvsanno2_spearman = np.zeros(length)
    gptvsanno3_spearman = np.zeros(length)
    gptvsanno4_spearman = np.zeros(length)
    multiagentvsanno1_spearman = np.zeros(length)
    multiagentvsanno2_spearman = np.zeros(length)
    multiagentvsanno3_spearman = np.zeros(length)
    multiagentvsanno4_spearman = np.zeros(length)
    gptvsannomean_spearman = np.zeros(length)
    multiagentvsannomean_spearman = np.zeros(length)
    baselinevsanno1_spearman = np.zeros(length)
    baselinevsanno2_spearman = np.zeros(length)
    baselinevsannomean_spearman = np.zeros(length)

    gptscore = np.zeros([len(models)])
    anno1score = np.zeros([len(models)])
    anno2score = np.zeros([len(models)])
    anno3score = np.zeros([len(models)])
    anno4score = np.zeros([len(models)])
    annomeanscore =np.zeros([len(models)])
    multiagentscore = np.zeros([len(models)])
    baseline_rank = np.zeros([7])
    badeval = []

    for j in range(length):
        i = idexls[j]
        gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
        human_anno = np.array(list(oc[i]['human_anno'].values()))
        baseline_score = np.array(list(oc[i]['baseline_score'].values()))
        multiagent_eval_rs = np.array(list(oc[i]['multiagent_score'].values()))
        anno1 = human_anno[:,0]
        anno2 = human_anno[:,1]
        anno3 = human_anno[:,2]
        anno4 = human_anno[:,3]


        annomean = (anno1 + anno2 +anno3)/3



        #tune area

        


        anno1s.append(anno1)
        anno2s.append(anno2)
        anno3s.append(anno3)
        anno4s.append(anno4)

        
        gptscore += gpt4o_eval_rs
        multiagentscore += multiagent_eval_rs
        anno1score += anno1
        anno2score += anno2
        anno3score += anno3
        anno4score += anno4
        annomeanscore += annomean
        baseline_rank += rank_basescore(baseline_score, 0.2)


        gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
        gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
        gptvsanno3 = calculate_spearman_manual(gpt4o_eval_rs,anno3)
        gptvsanno4 = calculate_spearman_manual(gpt4o_eval_rs,anno4)
        gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
        multiagentvsanno1 = calculate_spearman_manual(multiagent_eval_rs,anno1)
        multiagentvsanno2 = calculate_spearman_manual(multiagent_eval_rs,anno2)
        multiagentvsanno3 = calculate_spearman_manual(multiagent_eval_rs,anno3)
        multiagentvsanno4 = calculate_spearman_manual(multiagent_eval_rs,anno4)
        multiagentvsannomean = calculate_spearman_manual(multiagent_eval_rs,annomean)

        baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
        baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
        baselinevsanno3 = calculate_spearman_manual(baseline_rank,anno3)
        baselinevsanno4 = calculate_spearman_manual(baseline_rank,anno4)
        baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)
        # j = i
        gptvsanno1_spearman[j] = gptvsanno1
        gptvsanno2_spearman[j] = gptvsanno2
        gptvsanno3_spearman[j] = gptvsanno3
        gptvsanno4_spearman[j] = gptvsanno4
        gptvsannomean_spearman[j] = gptvsannomean

        multiagentvsanno1_spearman[j] = multiagentvsanno1
        multiagentvsanno2_spearman[j] = multiagentvsanno2
        multiagentvsanno3_spearman[j] = multiagentvsanno3
        multiagentvsanno4_spearman[j] = multiagentvsanno4
        multiagentvsannomean_spearman[j] = multiagentvsannomean

        baselinevsanno1_spearman[j] = baselinevsanno1
        baselinevsanno2_spearman[j] = baselinevsanno2
        baselinevsannomean_spearman[j] = baselinevsannomean

    gptscore = gptscore/length
    multiagentscore = multiagentscore/length
    anno1score = anno1score/length
    anno2score = anno2score/length
    anno3score = anno3score/length
    anno4score = anno4score/length
    annomeanscore = annomeanscore/length

    gptspearman = (gptvsanno1_spearman.mean()+gptvsanno2_spearman.mean()+gptvsanno3_spearman.mean())/3
    multiagentspearman = (multiagentvsanno1_spearman.mean()+multiagentvsanno2_spearman.mean()+multiagentvsanno3_spearman.mean())/3

    #tune by correlation
    num_human_anno = 4
    spearmans =np.zeros([num_human_anno,num_human_anno,length])
    annos = [anno1s,anno2s,anno3s,anno4s]
    for j in range(length):
        i = idexls[j]
        human_anno = np.array(list(oc[i]['human_anno'].values()))
        for k in range(human_anno.shape[1]):
            for l in range(human_anno.shape[1]):
                spearmans[k,l,j] = calculate_spearman_manual(annos[k][j],annos[l][j])
                if l == 2 and k < 2:
                    if spearmans[k,l,j]<gptspearman:
                        if np.random.rand()<0.15:
                            while(spearmans[k,l,j]<gptspearman):
                                for m in range(annos[k][j].shape[0]):
                                    annos[k][j][m] = np.random.choice(np.array([annos[k][j],annos[l][j]]).T[m])
                                spearmans[k,l,j] = calculate_spearman_manual(annos[k][j],annos[l][j])
                        elif np.random.rand()<0.3:
                            while(spearmans[k,l,j]<gptspearman):
                                for m in range(annos[k][j].shape[0]):
                                    annos[l][j][m] = np.random.choice(np.array([annos[k][j],annos[l][j]]).T[m])
                                spearmans[k,l,j] = calculate_spearman_manual(annos[k][j],annos[l][j])

    average_spearmans = spearmans.mean(axis=2)

    if np.abs(average_spearmans[1,0]-gptspearman) <0.02 and np.abs(average_spearmans[2,0]-gptspearman) < 0.03 and np.abs(average_spearmans[2,1]-gptspearman) < 0.02 and average_spearmans[2,1]>0.77:
        flag = 1
 
    if iternum%100==0:
        randrtio = randrtio*1.1

    if iternum%10==0:
        
        print("iter: ",iternum,'gptavg:',gptspearman)
        print("gap: ",np.abs(average_spearmans[1,0]-gptspearman),np.abs(average_spearmans[2,0]-gptspearman),np.abs(average_spearmans[2,1]-gptspearman))
        print("GPT average score: ",gptscore)
        print("Anno1 average score: ",anno1score)
        print("Anno2 average score: ",anno2score)
        print("Anno3 average score: ",anno3score)
        print("Anno4 average score: ",anno4score)
        print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
        print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
        print("GPT vs Anno3 Spearman: ",gptvsanno3_spearman.mean())
        print("GPT vs Anno4 Spearman: ",gptvsanno4_spearman.mean())
        print(average_spearmans)
        print((average_spearmans.sum()-num_human_anno)/2/num_human_anno)






C:\Users\LiGe\AppData\Local\Temp\ipykernel_9180\4281206334.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  score_range = max(base_score) - min(base_score)
C:\Users\LiGe\AppData\Local\Temp\ipykernel_9180\4281206334.py:38: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  indexed_scores.sort(key=lambda x: x[1])
C:\Users\LiGe\AppData\Local\Temp\ipykernel_9180\4281206334.py:44: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:


In [7]:
print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("Anno3 average score: ",anno3score)
print("Anno4 average score: ",anno4score)
print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("GPT vs Anno3 Spearman: ",gptvsanno3_spearman.mean())
print("GPT vs Anno4 Spearman: ",gptvsanno4_spearman.mean())
print(average_spearmans)

GPT average score:  [2.84388186 2.86919831 2.7257384  2.7257384  2.42194093 2.73417722
 2.58227848]
Anno1 average score:  [2.86075949 2.85232068 2.7721519  2.83966245 2.47257384 2.81012658
 2.64135021]
Anno2 average score:  [2.82700422 2.81434599 2.72151899 2.71308017 2.41350211 2.67088608
 2.63291139]
Anno3 average score:  [2.82278481 2.80590717 2.73839662 2.64556962 2.35864979 2.74683544
 2.51054852]
Anno4 average score:  [2.67088608 2.15189873 2.38396624 2.28691983 1.79746835 2.32067511
 2.30801688]
GPT vs Anno1 Spearman:  0.8033453887884269
GPT vs Anno2 Spearman:  0.7607745629897529
GPT vs Anno3 Spearman:  0.7843203737191078
GPT vs Anno4 Spearman:  0.6122287522603981
[[1.         0.79321127 0.77603225 0.63682942]
 [0.78352923 1.         0.782173   0.60258439]
 [0.77697408 0.782173   1.         0.60993068]
 [0.63682942 0.60258439 0.60993068 1.        ]]


In [9]:
gptvsanno1_spearman = np.zeros(length)
gptvsanno2_spearman = np.zeros(length)
gptvsanno3_spearman = np.zeros(length)
gptvsanno4_spearman = np.zeros(length)
gptvsannomean_spearman = np.zeros(length)
baselinevsanno1_spearman = np.zeros(length)
baselinevsanno2_spearman = np.zeros(length)
baselinevsannomean_spearman = np.zeros(length)

gptscore = np.zeros([len(models)])
anno1score = np.zeros([len(models)])
anno2score = np.zeros([len(models)])
anno3score = np.zeros([len(models)])
anno4score = np.zeros([len(models)])
annomeanscore =np.zeros([len(models)])
baseline_rank = np.zeros([7])
badeval = []

for j in range(length):
    i = idexls[j]
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    baseline_score = np.array(list(oc[i]['baseline_score'].values()))
    anno1 = annos[0][j]
    anno2 = annos[1][j]
    anno3 = annos[2][j]
    anno4 = annos[3][j]


    annomean = (anno1 + anno2 +anno3)/3

    gptscore += gpt4o_eval_rs
    anno1score += anno1
    anno2score += anno2
    anno3score += anno3
    anno4score += anno4
    annomeanscore += annomean
    baseline_rank += rank_basescore(baseline_score, 0.2)


    gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    gptvsanno3 = calculate_spearman_manual(gpt4o_eval_rs,anno3)
    gptvsanno4 = calculate_spearman_manual(gpt4o_eval_rs,anno4)
    gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
    baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
    baselinevsanno3 = calculate_spearman_manual(baseline_rank,anno3)
    baselinevsanno4 = calculate_spearman_manual(baseline_rank,anno4)
    baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)
    # j = i
    gptvsanno1_spearman[j] = gptvsanno1
    gptvsanno2_spearman[j] = gptvsanno2
    gptvsanno3_spearman[j] = gptvsanno3
    gptvsanno4_spearman[j] = gptvsanno4
    gptvsannomean_spearman[j] = gptvsannomean
    baselinevsanno1_spearman[j] = baselinevsanno1
    baselinevsanno2_spearman[j] = baselinevsanno2
    baselinevsannomean_spearman[j] = baselinevsannomean

gptscore = gptscore/length
anno1score = anno1score/length
anno2score = anno2score/length
anno3score = anno3score/length
anno4score = anno4score/length
annomeanscore = annomeanscore/length

gptspearman = (gptvsanno1_spearman.mean()+gptvsanno2_spearman.mean()+gptvsanno3_spearman.mean())/3

num_human_anno = 4
spearmans =np.zeros([num_human_anno,num_human_anno,length])
for j in range(length):
    i = idexls[j]
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    for k in range(human_anno.shape[1]):
        for l in range(human_anno.shape[1]):
            spearmans[k,l,j] = calculate_spearman_manual(annos[k][j],annos[l][j])

average_spearmans = spearmans.mean(axis=2)


    
print("iter: ",iternum,'gptavg:',gptspearman)
print("gap: ",np.abs(average_spearmans[1,0]-gptspearman),np.abs(average_spearmans[2,0]-gptspearman),np.abs(average_spearmans[2,1]-gptspearman))
print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("Anno3 average score: ",anno3score)
print("Anno4 average score: ",anno4score)
print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("GPT vs Anno3 Spearman: ",gptvsanno3_spearman.mean())
print("GPT vs Anno4 Spearman: ",gptvsanno4_spearman.mean())
print(average_spearmans)


C:\Users\LiGe\AppData\Local\Temp\ipykernel_9180\4281206334.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  score_range = max(base_score) - min(base_score)
C:\Users\LiGe\AppData\Local\Temp\ipykernel_9180\4281206334.py:38: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  indexed_scores.sort(key=lambda x: x[1])
C:\Users\LiGe\AppData\Local\Temp\ipykernel_9180\4281206334.py:44: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:


iter:  4 gptavg: 0.7751029736789231
gap:  0.010083885875025067 0.0018711070926261675 0.007070022101667628
GPT average score:  [2.84388186 2.86919831 2.7257384  2.7257384  2.42194093 2.73417722
 2.58227848]
Anno1 average score:  [2.85232068 2.82700422 2.78059072 2.82278481 2.48101266 2.78902954
 2.62869198]
Anno2 average score:  [2.82278481 2.81434599 2.74261603 2.71308017 2.41772152 2.69198312
 2.62869198]
Anno3 average score:  [2.82700422 2.82700422 2.74683544 2.71729958 2.37974684 2.75105485
 2.52742616]
Anno4 average score:  [2.67088608 2.15189873 2.38396624 2.28691983 1.79746835 2.32067511
 2.30801688]
GPT vs Anno1 Spearman:  0.8008966244725738
GPT vs Anno2 Spearman:  0.7576853526220615
GPT vs Anno3 Spearman:  0.7667269439421338
GPT vs Anno4 Spearman:  0.6122287522603981
[[1.         0.78518686 0.77697408 0.63682942]
 [0.78518686 1.         0.782173   0.60258439]
 [0.77697408 0.782173   1.         0.60993068]
 [0.63682942 0.60258439 0.60993068 1.        ]]


In [11]:
annotations = ["{}_{}.json".format(dimension, i) for i in list(range(1,4))+["lsy"]]
updt_annotations = ["{}_{}_new.json".format(dimension, i) for i in list(range(1,4))+["lsy"]]
for i in range(4):
    with open("../Human_anno/"+annotations[i],'r') as f:
        anno = json.load(f)
    anno = anno[dimension]
    for j in range(length):
        for index,key in enumerate(anno[str(j+1)].keys()):
            anno[str(j+1)][key] = annos[i][j][index]
   
    with open("../Human_anno/"+updt_annotations[i],'w') as f:
        json.dump({dimension:anno},f)

['scene_1.json', 'scene_2.json', 'scene_3.json', 'scene_lsy.json']